# Gemini 텍스트 생성

* 모델 종류: https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko&_gl=1

In [7]:
import os
from dotenv import load_dotenv

# 환경변수 로드 및 GEMINI_API_KEY 가져오기
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

## 텍스트 전용 입력에서 텍스트 생성

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["How does AI work?"])
print(response.text)

## 텍스트 및 이미지 입력에서 텍스트 생성

* Gemini API는 텍스트와 미디어 파일을 결합한 멀티모달 입력을 지원합니다. 다음 예는 텍스트 및 이미지 입력에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from PIL import Image
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

image = Image.open("/path/to/organ.png")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[image, "Tell me about this instrument"])
print(response.text)

## 텍스트 스트림 생성

* 기본적으로 모델은 전체 텍스트 생성 프로세스를 완료한 후에 응답을 반환합니다. 전체 결과를 기다리지 않고 대신 스트리밍을 사용하여 부분 결과를 처리하면 더 빠른 상호작용을 실행할 수 있습니다.
* 다음 예에서는 streamGenerateContent 메서드를 사용하여 스트리밍을 구현하여 텍스트 전용 입력 프롬프트에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"])
for chunk in response:
    print(chunk.text, end="")

## 채팅 대화 만들기

* Gemini SDK를 사용하면 여러 번의 질문과 응답을 수집할 수 있으므로 사용자가 점진적으로 답변을 찾거나 여러 부분으로 구성된 문제와 관련하여 도움을 받을 수 있습니다. 이 SDK 기능은 대화 기록을 추적하는 인터페이스를 제공하지만, 백그라운드에서는 동일한 generateContent 메서드를 사용하여 응답을 만듭니다.

* 다음 코드 예는 기본 채팅 구현을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

* 다음 예와 같이 채팅과 함께 스트리밍을 사용할 수도 있습니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text, end="")
response = chat.send_message_stream("How many paws are in my house?")
for chunk in response:
    print(chunk.text, end="")
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

## 텍스트 생성 구성

* 모델에 전송하는 모든 프롬프트에는 모델이 응답을 생성하는 방식을 제어하는 매개변수가 포함됩니다. GenerationConfig를 사용하여 이러한 매개변수를 구성할 수 있습니다. 매개변수를 구성하지 않으면 모델은 기본 옵션을 사용하며 이는 모델에 따라 다를 수 있습니다.

* 다음 예는 사용 가능한 여러 옵션을 구성하는 방법을 보여줍니다.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

## 시스템 안내 추가

* 시스템 안내를 사용하면 특정 요구사항 및 사용 사례에 따라 모델의 동작을 조정할 수 있습니다.

* 모델에 시스템 안내를 제공하면 작업을 이해하고, 보다 맞춤설정된 대답을 생성하고, 모델과의 전체 사용자 상호작용에 대한 특정 가이드라인을 준수하기 위한 추가 컨텍스트를 모델에 제공할 수 있습니다. 최종 사용자가 제공하는 프롬프트와 별도로 시스템 안내를 설정하여 제품 수준 동작을 지정할 수도 있습니다.

* 모델을 초기화할 때 시스템 안내를 설정할 수 있습니다.

In [ ]:
sys_instruct="You are a cat. Your name is Neko."
client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["your prompt here"]
)

## 소설 제목 TEST

### gemini-2.0-flash

In [20]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = "당신은 창의적이고 독창적인 소설 제목 작가입니다. 주어진 장르를 기반으로 적절한 소설 제목을 만들어보세요."

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"

prompt = f"""
## 소설 장르: {genre}

**소설 제목**
"""

response = model.generate_content(prompt)
print(response.text)

## 판타지, 액션, 일상물 장르 소설 제목

여기 몇 가지 다양한 분위기의 제목 옵션을 제시합니다. 어떤 톤과 분위기를 선호하는지에 따라 선택하시면 좋을 것 같습니다.

**1. 서정적이고 신비로운 느낌:**

*   별빛 아래 핀 아리아: 용의 심장을 가진 제빵사
*   일상의 마법, 그림자 속의 검: 평범한 모험가의 노래
*   꿈결의 레시피, 강철의 향기: 이세계 식당의 비밀
*   달콤한 위협: 마법 학교의 평화로운 암살자

**2. 액션과 모험을 강조:**

*   빵 굽는 용사: 던전에서 피어나는 맛
*   오늘도 던전 청소합니다: 힐링은 덤!
*   마법은 거들 뿐, 주먹이 최고: 이세계 무림 생존기
*   흔한 요리사의 숨겨진 힘: 몬스터는 나의 식재료!

**3. 일상적인 재미와 유머 강조:**

*   마법? 액션? 됐고, 일단 빵부터!
*   용사 은퇴 후 제2의 삶: 힐링 밥상 차리기
*   평범한 직장인의 판타지 라이프: 야근은 어디에나
*   이세계에서 살아남기: 삼시세끼 자급자족

**4. 독특하고 개성적인 느낌:**

*   마나 과다 냥냥 펀치!
*   정령왕 엘프씨, 오늘은 뭘 구워드릴까요?
*   회색 마법사의 달콤쌉싸름한 디저트 마법
*   [시스템] 요리 스킬을 획득하셨습니다! (feat. 몬스터)

**선택에 도움이 될 추가 질문:**

*   주인공의 성격은 어떤가요? (ex: 쾌활, 냉철, 허당 등)
*   소설의 주된 분위기는 어떤가요? (ex: 밝고 코믹, 진지하고 어두움 등)
*   강조하고 싶은 요소는 무엇인가요? (ex: 요리, 액션, 성장 등)

이 질문들에 대한 답변을 알려주시면 더욱 맞춤 Title을 만들어 드릴 수 있습니다.


## 소설 세계관 TEST

### gemini-2.0-flash

In [17]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = "당신은 창의적이고 독창적인 소설 세계관 작가입니다. 주어진 장르, 제목을 기반으로 소설 세계관을 만들어보세요."

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

**소설 세계관**
"""

response = model.generate_content(prompt)
print(response.text)

## 소설 세계관: 괴식식당

**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한

### gemini-1.5-pro

In [18]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = "당신은 창의적이고 독창적인 소설 세계관 작가입니다. 주어진 장르, 제목을 기반으로 소설 세계관을 만들어보세요."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

**소설 세계관**
"""

response = model.generate_content(prompt)
print(response.text)

## 괴식식당 세계관 설정

**1. 세계관 개요:**

현대 사회와 유사하지만, "미식 차원"이라는 또 다른 차원과 연결된 세계.  미식 차원은 온갖 기상천외한 식재료와 요리가 존재하는 곳으로, 일부 특별한 사람들만이 "미식 문"을 통해 이 차원에 접근할 수 있다.  "괴식식당"은 미식 차원의 재료를 사용하여 특별한 요리를 제공하는 식당들을 일컫는 말이다.  이 식당들은 단순한 식당을 넘어, 미식 모험가들의 거점이자, 때로는 차원 괴물 침략의 최전선이 되기도 한다.

**2. 미식 차원:**

* **다양한 환경:** 사탕으로 만들어진 숲, 치즈로 이루어진 산맥, 육수가 흐르는 강 등 상상을 초월하는 다양한 환경이 존재한다.
* **괴식재료:** 살아 움직이는 채소, 노래하는 과일, 감정을 가진 고기 등 기이하고 특별한 능력을 지닌 식재료들이 가득하다.
* **차원 괴물:** 식재료를 노리는 괴물들이 존재하며, 이들은 미식 문을 통해 현실 세계로 넘어와 혼란을 야기하기도 한다.
* **미식 정령:** 미식 차원을 수호하는 정령들이 존재하며,  일부 미식가들과 계약을 맺고 힘을 빌려주기도 한다.

**3. 괴식식당:**

* **미식 셰프:** 미식 문을 통해 재료를 구하고, 특별한 레시피로 요리하는 능력자들.  뛰어난 요리 실력은 물론, 차원 괴물과 싸울 수 있는 전투 능력도 필요하다.
* **특별 메뉴:**  재료 본연의 힘을 끌어올린 요리는 맛뿐만 아니라, 먹는 사람에게 특별한 능력을 부여하기도 한다. (예: 힘 증가, 재생 능력, 비행 능력 등)
* **단골손님:** 괴식에 중독된 미식가, 능력 향상을 원하는 모험가, 괴물 퇴치를 의뢰하는 정부 기관원 등 다양한 손님들이 찾아온다.
* **숨겨진 기능:** 일부 괴식식당은 미식 차원으로 통하는 비밀 통로를 숨기고 있거나, 괴물 퇴치를 위한 특수 장비를 보유하고 있다.

**4. 주요 세력:**

* **미식가 길드:** 미식 차원 탐험과 괴식 연구를 장려하는 단체. 미식 셰프 자격증을 발급하고, 괴물 퇴

## 소설 줄거리 Test

### gemini-2.0-flash

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = "당신은 창의적이고 독창적인 소설 줄거리 작가입니다. 주어진 장르, 제목, 세계관을 기반으로 소설 줄거리를 만들어보세요."

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

## 소설 세계관: {worldview}

**소설 줄거리**
"""

response = model.generate_content(prompt)
print(response.text)

## 소설 줄거리: 괴식식당

**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 

## 소설 등장인물 TEST

### gemini-2.0-flash

In [21]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 주어진 장르, 제목, 세계관, 줄거리를 기반으로 소설 등장인물을 만들어보세요.
단, 등장인물의 설정은 이름, 성별, 나이, 역할, 직업, 프로필로 구성되어야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""
synopsis = """
**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 탐험하며, 엘프, 드워프, 수인 등 다양한 종족들과 협력하거나 대립한다. 강철수는 검술 실력과 요리 지식을 활용하여 적들을 물리치고, 린은 뛰어난 후각과 미각으로 숨겨진 단서를 찾아낸다.

**5. 일상과 성장:**

위험한 모험 속에서도 괴식식당은 평화로운 일상을 유지한다. 강철수는 새로운 괴식 레시피를 개발하고, 린은 손님들에게 따뜻한 미소를 짓는다. 식당을 찾아오는 손님들은 괴식을 통해 위로를 받거나 희망을 얻고, 강철수와 린은 그들의 삶에 작은 변화를 가져다준다. 강철수는 린을 통해 과거의 상처를 치유하고, 린은 강철수를 통해 세상을 향해 나아갈 용기를 얻는다.
...
**7. 결말:**

사건이 해결되고, 미식령에는 다시 평화가 찾아온다. 강철수는 괴식식당을 계속 운영하며, 린과 함께 새로운 괴식 레시피를 개발하고 손님들에게 행복을 선사한다. 괴식식당은 미식령의 명소가 되고, 사람들은 괴식을 통해 서로를 이해하고 사랑하며 살아간다. 강철수는 과거의 상처를 극복하고 새로운 삶을 살아가며, 린은 미식령 최고의 요리사가 되기 위해 노력한다. 괴식식당은 오늘도 맛있는 괴식과 따뜻한 이야기로 가득하다.
"""

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

## 소설 세계관: {worldview}

## 소설 줄거리: {synopsis}

**소설 등장인물**
"""

response = model.generate_content(prompt)
print(response.text)

## 소설 등장인물

### 1. 강철수

*   **성별:** 남
*   **나이:** 40대 초반
*   **역할:** 괴식식당 주인, 퇴물 검사
*   **직업:** 요리사, 전직 검사
*   **프로필:**
    *   과거 뛰어난 실력을 자랑했던 검사였으나, 모종의 사건으로 인해 검을 놓았다.
    *   얼굴에 커다란 흉터가 있으며, 과묵하고 냉정한 인상을 준다.
    *   요리 실력이 뛰어나며, 특히 괴상한 식재료를 다루는 데 능숙하다.
    *   과거의 트라우마 때문에 세상과 단절된 채 살아가지만, 속으로는 따뜻한 마음을 품고 있다.
    *   무뚝뚝하지만 린을 챙기는 모습에서 따뜻함이 묻어 나온다.
    *   주 무기는 식칼이며, 검술 실력은 녹슬지 않았다.

### 2. 린

*   **성별:** 여
*   **나이:** 10대 후반 (묘족 기준)
*   **역할:** 괴식식당 직원, 강철수의 제자
*   **직업:** 요리사 견습생
*   **프로필:**
    *   묘족 출신으로, 뛰어난 후각과 미각을 지니고 있다.
    *   희귀병 때문에 평범한 음식을 먹을 수 없었지만, 강철수의 괴식을 통해 미각을 되찾았다.
    *   밝고 활발하며, 호기심이 많다.
    *   강철수를 스승이자 아버지처럼 따르며, 그의 요리 실력을 배우고 싶어 한다.
    *   긍정적이고 낙천적인 성격으로, 주변 사람들에게 긍정적인 에너지를 불어넣는다.
    *   동물과 교감하는 능력이 뛰어나다.

### 3. 장로

*   **성별:** 남
*   **나이:** 100대 후반
*   **역할:** 미식령 장로
*   **직업:** 은퇴
*   **프로필:**
    *   미식령의 깊은 산속에 은거하며, 묘족을 통솔하는 장로이다.
    *   나이가 많지만, 카리스마를 잃지 않았다.
    *   강철수와 린에게 도움을 주는 조력자이다.
    *   박식하고 현명하며, 미식령의 역사와 괴식에 대한 해박한 지식을 가지고 있다.

## 소설 등장인물 추가

### gemini-2.0-flash

In [22]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 주어진 장르, 제목, 세계관, 줄거리, 기존의 등장인물을 기반으로 새로운 소설 등장인물을 만들어보세요.
단, 등장인물의 설정은 이름, 성별, 나이, 역할, 직업, 프로필로 구성되어야 합니다.
또한, 기존의 등장인물와의 상호작용이 자연스럽도록 설정해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""
synopsis = """
**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 탐험하며, 엘프, 드워프, 수인 등 다양한 종족들과 협력하거나 대립한다. 강철수는 검술 실력과 요리 지식을 활용하여 적들을 물리치고, 린은 뛰어난 후각과 미각으로 숨겨진 단서를 찾아낸다.

**5. 일상과 성장:**

위험한 모험 속에서도 괴식식당은 평화로운 일상을 유지한다. 강철수는 새로운 괴식 레시피를 개발하고, 린은 손님들에게 따뜻한 미소를 짓는다. 식당을 찾아오는 손님들은 괴식을 통해 위로를 받거나 희망을 얻고, 강철수와 린은 그들의 삶에 작은 변화를 가져다준다. 강철수는 린을 통해 과거의 상처를 치유하고, 린은 강철수를 통해 세상을 향해 나아갈 용기를 얻는다.
...
**7. 결말:**

사건이 해결되고, 미식령에는 다시 평화가 찾아온다. 강철수는 괴식식당을 계속 운영하며, 린과 함께 새로운 괴식 레시피를 개발하고 손님들에게 행복을 선사한다. 괴식식당은 미식령의 명소가 되고, 사람들은 괴식을 통해 서로를 이해하고 사랑하며 살아간다. 강철수는 과거의 상처를 극복하고 새로운 삶을 살아가며, 린은 미식령 최고의 요리사가 되기 위해 노력한다. 괴식식당은 오늘도 맛있는 괴식과 따뜻한 이야기로 가득하다.
"""
characters = """
### 1. 강철수

*   **성별:** 남
*   **나이:** 40대 초반
*   **역할:** 괴식식당 주인, 퇴물 검사
*   **직업:** 요리사, 전직 검사
*   **프로필:**
    *   과거 뛰어난 실력을 자랑했던 검사였으나, 모종의 사건으로 인해 검을 놓았다.
    *   얼굴에 커다란 흉터가 있으며, 과묵하고 냉정한 인상을 준다.
    *   요리 실력이 뛰어나며, 특히 괴상한 식재료를 다루는 데 능숙하다.
    *   과거의 트라우마 때문에 세상과 단절된 채 살아가지만, 속으로는 따뜻한 마음을 품고 있다.
    *   무뚝뚝하지만 린을 챙기는 모습에서 따뜻함이 묻어 나온다.
    *   주 무기는 식칼이며, 검술 실력은 녹슬지 않았다.

### 2. 린

*   **성별:** 여
*   **나이:** 10대 후반 (묘족 기준)
*   **역할:** 괴식식당 직원, 강철수의 제자
*   **직업:** 요리사 견습생
*   **프로필:**
    *   묘족 출신으로, 뛰어난 후각과 미각을 지니고 있다.
    *   희귀병 때문에 평범한 음식을 먹을 수 없었지만, 강철수의 괴식을 통해 미각을 되찾았다.
...
    *   과거 강철수와 깊은 관계가 있었으며, 그에게 복수심을 품고 있다.
    *   정체를 숨기고 활동하며, 그의 목적과 과거는 미스터리에 싸여 있다.
    *   지팡이, 마법, 괴수 등 모든 것이 무기이다.
"""

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

## 소설 세계관: {worldview}

## 소설 줄거리: {synopsis}

## 기존 소설 등장인물: {characters}

**새로운 소설 등장인물**
"""

response = model.generate_content(prompt)
print(response.text)

## 새로운 등장인물: 

### 3. 멜리아 (Melia)

*   **성별:** 여
*   **나이:** 겉보기 20대 중반 (실제 나이 불명)
*   **역할:** 떠돌이 약초꾼 겸 괴식 감별사
*   **직업:** 약초꾼, 괴식 감별사, 자칭 '미식 평론가'
*   **프로필:**
    *   엘프와 인간의 혼혈로, 숲과 도시에 모두 익숙하다. 
    *   밝고 쾌활한 성격으로, 처음 보는 사람에게도 쉽게 말을 건다. 호기심이 왕성하며, 새로운 괴식을 맛보는 것을 즐긴다.
    *   뛰어난 약초 지식을 가지고 있으며, 특히 괴식 재료의 효능과 부작용에 대해 해박하다. 괴식 재료의 독성을 중화하거나, 맛을 개선하는 비법을 알고 있다.
    *   자신을 '미식 평론가'라 칭하며, 괴식을 맛본 후 신랄하면서도 재치 있는 평가를 내린다. 하지만 그녀의 평가는 때로는 너무 솔직해서 사람들을 당황하게 만들기도 한다.
    *   돈을 밝히는 속물적인 면모도 있지만, 어려운 사람을 보면 그냥 지나치지 못하는 따뜻한 마음씨를 가지고 있다.
    *   등에는 커다란 배낭을 메고 다니며, 그 안에는 약초, 조미료, 요리 도구 등 다양한 물건들이 들어있다.
    *   린의 요리 실력에 감탄하며, 그녀를 '천재 요리사'라 칭찬한다. 린에게 새로운 괴식 재료를 소개하거나, 요리 비법을 알려주기도 한다.
    *   강철수의 과거에 대해 궁금해하며, 그의 굳게 닫힌 마음을 열려고 노력한다. 때로는 강철수를 짓궂게 놀리기도 하지만, 그의 아픔을 이해하고 위로하려 한다.

**기존 등장인물과의 상호작용:**

*   **강철수와의 상호작용:**
    *   처음에는 강철수의 무뚝뚝한 태도에 당황하지만, 그의 요리 실력과 숨겨진 따뜻함에 호감을 느낀다.
    *   강철수에게 새로운 괴식 재료를 소개하거나, 요리 비법을 알려주며 식당 운영에 도움을 준다.
    *   강철수의 과거에 대해 캐묻거나, 그의 트라우마를 자극하는 발언을 하기도 한다. 하지만 진

## 소설 초안 TEST

### gemini-2.0-flash

In [23]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 
주어진 장르, 제목, 세계관, 줄거리, 등장인물을 기반으로 소설의 초안을 작성해야 합니다.
장르의 분위기에 맞게 전개하되, 500-1000자 정도의 내용을 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""
synopsis = """
**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 탐험하며, 엘프, 드워프, 수인 등 다양한 종족들과 협력하거나 대립한다. 강철수는 검술 실력과 요리 지식을 활용하여 적들을 물리치고, 린은 뛰어난 후각과 미각으로 숨겨진 단서를 찾아낸다.

**5. 일상과 성장:**

위험한 모험 속에서도 괴식식당은 평화로운 일상을 유지한다. 강철수는 새로운 괴식 레시피를 개발하고, 린은 손님들에게 따뜻한 미소를 짓는다. 식당을 찾아오는 손님들은 괴식을 통해 위로를 받거나 희망을 얻고, 강철수와 린은 그들의 삶에 작은 변화를 가져다준다. 강철수는 린을 통해 과거의 상처를 치유하고, 린은 강철수를 통해 세상을 향해 나아갈 용기를 얻는다.
...
**7. 결말:**

사건이 해결되고, 미식령에는 다시 평화가 찾아온다. 강철수는 괴식식당을 계속 운영하며, 린과 함께 새로운 괴식 레시피를 개발하고 손님들에게 행복을 선사한다. 괴식식당은 미식령의 명소가 되고, 사람들은 괴식을 통해 서로를 이해하고 사랑하며 살아간다. 강철수는 과거의 상처를 극복하고 새로운 삶을 살아가며, 린은 미식령 최고의 요리사가 되기 위해 노력한다. 괴식식당은 오늘도 맛있는 괴식과 따뜻한 이야기로 가득하다.
"""
characters = """
### 1. 강철수

*   **성별:** 남
*   **나이:** 40대 초반
*   **역할:** 괴식식당 주인, 퇴물 검사
*   **직업:** 요리사, 전직 검사
*   **프로필:**
    *   과거 뛰어난 실력을 자랑했던 검사였으나, 모종의 사건으로 인해 검을 놓았다.
    *   얼굴에 커다란 흉터가 있으며, 과묵하고 냉정한 인상을 준다.
    *   요리 실력이 뛰어나며, 특히 괴상한 식재료를 다루는 데 능숙하다.
    *   과거의 트라우마 때문에 세상과 단절된 채 살아가지만, 속으로는 따뜻한 마음을 품고 있다.
    *   무뚝뚝하지만 린을 챙기는 모습에서 따뜻함이 묻어 나온다.
    *   주 무기는 식칼이며, 검술 실력은 녹슬지 않았다.

### 2. 린

*   **성별:** 여
*   **나이:** 10대 후반 (묘족 기준)
*   **역할:** 괴식식당 직원, 강철수의 제자
*   **직업:** 요리사 견습생
*   **프로필:**
    *   묘족 출신으로, 뛰어난 후각과 미각을 지니고 있다.
    *   희귀병 때문에 평범한 음식을 먹을 수 없었지만, 강철수의 괴식을 통해 미각을 되찾았다.
...
    *   과거 강철수와 깊은 관계가 있었으며, 그에게 복수심을 품고 있다.
    *   정체를 숨기고 활동하며, 그의 목적과 과거는 미스터리에 싸여 있다.
    *   지팡이, 마법, 괴수 등 모든 것이 무기이다.

### 3. 멜리아 (Melia)

*   **성별:** 여
*   **나이:** 겉보기 20대 중반 (실제 나이 불명)
*   **역할:** 떠돌이 약초꾼 겸 괴식 감별사
*   **직업:** 약초꾼, 괴식 감별사, 자칭 '미식 평론가'
*   **프로필:**
    *   엘프와 인간의 혼혈로, 숲과 도시에 모두 익숙하다. 
    *   밝고 쾌활한 성격으로, 처음 보는 사람에게도 쉽게 말을 건다. 호기심이 왕성하며, 새로운 괴식을 맛보는 것을 즐긴다.
    *   뛰어난 약초 지식을 가지고 있으며, 특히 괴식 재료의 효능과 부작용에 대해 해박하다. 괴식 재료의 독성을 중화하거나, 맛을 개선하는 비법을 알고 있다.
    *   자신을 '미식 평론가'라 칭하며, 괴식을 맛본 후 신랄하면서도 재치 있는 평가를 내린다. 하지만 그녀의 평가는 때로는 너무 솔직해서 사람들을 당황하게 만들기도 한다.
    *   돈을 밝히는 속물적인 면모도 있지만, 어려운 사람을 보면 그냥 지나치지 못하는 따뜻한 마음씨를 가지고 있다.
    *   등에는 커다란 배낭을 메고 다니며, 그 안에는 약초, 조미료, 요리 도구 등 다양한 물건들이 들어있다.
    *   린의 요리 실력에 감탄하며, 그녀를 '천재 요리사'라 칭찬한다. 린에게 새로운 괴식 재료를 소개하거나, 요리 비법을 알려주기도 한다.
    *   강철수의 과거에 대해 궁금해하며, 그의 굳게 닫힌 마음을 열려고 노력한다. 때로는 강철수를 짓궂게 놀리기도 하지만, 그의 아픔을 이해하고 위로하려 한다.
"""

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

## 소설 세계관: {worldview}

## 소설 줄거리: {synopsis}

## 소설 등장인물: {characters}

**소설 초안**
"""

response = model.generate_content(prompt)
print(response.text)

## 괴식식당

**프롤로그**

미식령 변두리, 척박한 돌산 자락에 덩그러니 놓인 ‘괴식식당’. 낡은 나무 간판은 군데군데 칠이 벗겨져 있었고, 삐뚤빼뚤한 글씨로 ‘손님, 뭘 먹어도 책임 안 짐’이라고 적혀 있었다. 그 아래, 녹슨 풍경이 바람에 흔들릴 때마다 묘한 소리를 냈다. 식당 안은 어두컴컴했다. 

주인, 강철수는 묵묵히 칼을 갈고 있었다. 흉터투성이의 얼굴은 그림자에 가려져 잘 보이지 않았다. 한때 그는 미식령 최고의 검사 중 하나였지만, 지금은 그저 퇴물일 뿐이었다. 과거의 영광은 빛바랜 훈장처럼 그의 마음속 깊은 곳에 묻혀 있었다. 
그의 손에 들린 것은 평범한 식칼이 아니었다. 검날은 검푸른 빛을 띠고 있었고, 칼등에는 희미하게 빛나는 문양이 새겨져 있었다. 그는 이 칼로 괴상한 식재료들을 손질하며 하루하루를 보냈다. 독개구리의 맹독을 제거하고, 지옥 벌레의 껍질을 벗겨내고, 심연 해삼의 끈적한 점액을 닦아냈다.

**1. 묘족 소녀, 린**

어느 날, 식당의 문이 조용히 열렸다. 손님은 묘족 소녀, 린이었다. 그녀는 커다란 눈망울을 빛내며 조심스럽게 식당 안으로 들어왔다. 그녀의 옷은 먼지로 더럽혀져 있었고, 얼굴은 창백했다. 린은 희귀한 병을 앓고 있었다. 평범한 음식을 먹을 수 없는 병이었다. 그녀는 미식령을 떠돌아다니며 자신의 병을 치료할 음식을 찾아 헤매고 있었다.

"어서 오세요." 강철수는 무뚝뚝하게 말했다. 그의 목소리는 쇠처럼 차가웠다.
린은 주눅 든 듯 고개를 숙였다. "저... 괴식을 먹을 수 있다는 소문을 듣고 왔습니다."
강철수는 린을 가만히 쳐다보았다. "괴식은 위험하다. 잘못 먹으면 목숨을 잃을 수도 있어."
"알고 있습니다. 하지만... 저는 평범한 음식을 먹을 수 없습니다. 괴식이라도 먹을 수 있다면..." 린은 간절한 눈빛으로 강철수를 바라보았다.

강철수는 잠시 생각에 잠겼다. 그리고는 묵묵히 칼을 내려놓고 주방으로 향했다. 잠시 후, 그는 김이 모락모락 나는 수프 한 그릇을 들고 나왔다.
"독개구리 수프다

## 소설 다음화 작성 TEST

### gemini-2.0-flash

In [24]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 창의적이고 독창적인 소설 작가입니다. 
주어진 장르, 제목, 세계관, 줄거리, 등장인물, 소설 이전화를 기반으로 소설의 다음화를 작성해야 합니다.
장르의 분위기에 맞게 전개하되, 500-1000자 정도의 내용을 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""
synopsis = """
**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 탐험하며, 엘프, 드워프, 수인 등 다양한 종족들과 협력하거나 대립한다. 강철수는 검술 실력과 요리 지식을 활용하여 적들을 물리치고, 린은 뛰어난 후각과 미각으로 숨겨진 단서를 찾아낸다.

**5. 일상과 성장:**

위험한 모험 속에서도 괴식식당은 평화로운 일상을 유지한다. 강철수는 새로운 괴식 레시피를 개발하고, 린은 손님들에게 따뜻한 미소를 짓는다. 식당을 찾아오는 손님들은 괴식을 통해 위로를 받거나 희망을 얻고, 강철수와 린은 그들의 삶에 작은 변화를 가져다준다. 강철수는 린을 통해 과거의 상처를 치유하고, 린은 강철수를 통해 세상을 향해 나아갈 용기를 얻는다.
...
**7. 결말:**

사건이 해결되고, 미식령에는 다시 평화가 찾아온다. 강철수는 괴식식당을 계속 운영하며, 린과 함께 새로운 괴식 레시피를 개발하고 손님들에게 행복을 선사한다. 괴식식당은 미식령의 명소가 되고, 사람들은 괴식을 통해 서로를 이해하고 사랑하며 살아간다. 강철수는 과거의 상처를 극복하고 새로운 삶을 살아가며, 린은 미식령 최고의 요리사가 되기 위해 노력한다. 괴식식당은 오늘도 맛있는 괴식과 따뜻한 이야기로 가득하다.
"""
characters = """
### 1. 강철수

*   **성별:** 남
*   **나이:** 40대 초반
*   **역할:** 괴식식당 주인, 퇴물 검사
*   **직업:** 요리사, 전직 검사
*   **프로필:**
    *   과거 뛰어난 실력을 자랑했던 검사였으나, 모종의 사건으로 인해 검을 놓았다.
    *   얼굴에 커다란 흉터가 있으며, 과묵하고 냉정한 인상을 준다.
    *   요리 실력이 뛰어나며, 특히 괴상한 식재료를 다루는 데 능숙하다.
    *   과거의 트라우마 때문에 세상과 단절된 채 살아가지만, 속으로는 따뜻한 마음을 품고 있다.
    *   무뚝뚝하지만 린을 챙기는 모습에서 따뜻함이 묻어 나온다.
    *   주 무기는 식칼이며, 검술 실력은 녹슬지 않았다.

### 2. 린

*   **성별:** 여
*   **나이:** 10대 후반 (묘족 기준)
*   **역할:** 괴식식당 직원, 강철수의 제자
*   **직업:** 요리사 견습생
*   **프로필:**
    *   묘족 출신으로, 뛰어난 후각과 미각을 지니고 있다.
    *   희귀병 때문에 평범한 음식을 먹을 수 없었지만, 강철수의 괴식을 통해 미각을 되찾았다.
...
    *   과거 강철수와 깊은 관계가 있었으며, 그에게 복수심을 품고 있다.
    *   정체를 숨기고 활동하며, 그의 목적과 과거는 미스터리에 싸여 있다.
    *   지팡이, 마법, 괴수 등 모든 것이 무기이다.

### 3. 멜리아 (Melia)

*   **성별:** 여
*   **나이:** 겉보기 20대 중반 (실제 나이 불명)
*   **역할:** 떠돌이 약초꾼 겸 괴식 감별사
*   **직업:** 약초꾼, 괴식 감별사, 자칭 '미식 평론가'
*   **프로필:**
    *   엘프와 인간의 혼혈로, 숲과 도시에 모두 익숙하다. 
    *   밝고 쾌활한 성격으로, 처음 보는 사람에게도 쉽게 말을 건다. 호기심이 왕성하며, 새로운 괴식을 맛보는 것을 즐긴다.
    *   뛰어난 약초 지식을 가지고 있으며, 특히 괴식 재료의 효능과 부작용에 대해 해박하다. 괴식 재료의 독성을 중화하거나, 맛을 개선하는 비법을 알고 있다.
    *   자신을 '미식 평론가'라 칭하며, 괴식을 맛본 후 신랄하면서도 재치 있는 평가를 내린다. 하지만 그녀의 평가는 때로는 너무 솔직해서 사람들을 당황하게 만들기도 한다.
    *   돈을 밝히는 속물적인 면모도 있지만, 어려운 사람을 보면 그냥 지나치지 못하는 따뜻한 마음씨를 가지고 있다.
    *   등에는 커다란 배낭을 메고 다니며, 그 안에는 약초, 조미료, 요리 도구 등 다양한 물건들이 들어있다.
    *   린의 요리 실력에 감탄하며, 그녀를 '천재 요리사'라 칭찬한다. 린에게 새로운 괴식 재료를 소개하거나, 요리 비법을 알려주기도 한다.
    *   강철수의 과거에 대해 궁금해하며, 그의 굳게 닫힌 마음을 열려고 노력한다. 때로는 강철수를 짓궂게 놀리기도 하지만, 그의 아픔을 이해하고 위로하려 한다.
"""
first_chapter = """
**프롤로그**

미식령 변두리, 척박한 돌산 자락에 덩그러니 놓인 ‘괴식식당’. 낡은 나무 간판은 군데군데 칠이 벗겨져 있었고, 삐뚤빼뚤한 글씨로 ‘손님, 뭘 먹어도 책임 안 짐’이라고 적혀 있었다. 그 아래, 녹슨 풍경이 바람에 흔들릴 때마다 묘한 소리를 냈다. 식당 안은 어두컴컴했다. 

주인, 강철수는 묵묵히 칼을 갈고 있었다. 흉터투성이의 얼굴은 그림자에 가려져 잘 보이지 않았다. 한때 그는 미식령 최고의 검사 중 하나였지만, 지금은 그저 퇴물일 뿐이었다. 과거의 영광은 빛바랜 훈장처럼 그의 마음속 깊은 곳에 묻혀 있었다. 
그의 손에 들린 것은 평범한 식칼이 아니었다. 검날은 검푸른 빛을 띠고 있었고, 칼등에는 희미하게 빛나는 문양이 새겨져 있었다. 그는 이 칼로 괴상한 식재료들을 손질하며 하루하루를 보냈다. 독개구리의 맹독을 제거하고, 지옥 벌레의 껍질을 벗겨내고, 심연 해삼의 끈적한 점액을 닦아냈다.

**1. 묘족 소녀, 린**

어느 날, 식당의 문이 조용히 열렸다. 손님은 묘족 소녀, 린이었다. 그녀는 커다란 눈망울을 빛내며 조심스럽게 식당 안으로 들어왔다. 그녀의 옷은 먼지로 더럽혀져 있었고, 얼굴은 창백했다. 린은 희귀한 병을 앓고 있었다. 평범한 음식을 먹을 수 없는 병이었다. 그녀는 미식령을 떠돌아다니며 자신의 병을 치료할 음식을 찾아 헤매고 있었다.

"어서 오세요." 강철수는 무뚝뚝하게 말했다. 그의 목소리는 쇠처럼 차가웠다.
린은 주눅 든 듯 고개를 숙였다. "저... 괴식을 먹을 수 있다는 소문을 듣고 왔습니다."
강철수는 린을 가만히 쳐다보았다. "괴식은 위험하다. 잘못 먹으면 목숨을 잃을 수도 있어."
"알고 있습니다. 하지만... 저는 평범한 음식을 먹을 수 없습니다. 괴식이라도 먹을 수 있다면..." 린은 간절한 눈빛으로 강철수를 바라보았다.

강철수는 잠시 생각에 잠겼다. 그리고는 묵묵히 칼을 내려놓고 주방으로 향했다. 잠시 후, 그는 김이 모락모락 나는 수프 한 그릇을 들고 나왔다.
"독개구리 수프다. 맹독은 제거했지만, 마비 성분이 남아 있을 수 있으니 조심해서 먹어."
린은 떨리는 손으로 수프를 받아 들었다. 그녀는 조심스럽게 수프를 한 모금 마셨다. 순간, 그녀의 얼굴이 일그러졌다. 혀가 마비되는 듯한 감각과 함께 온몸에 전기가 흐르는 듯한 고통이 느껴졌다. 그러나 동시에, 그녀는 지금까지 느껴보지 못했던 강렬한 맛을 느꼈다.

"맛...있어..." 린은 눈물을 글썽이며 말했다. 그녀는 수프를 남김없이 비웠다. 그 순간, 린은 잃어버렸던 미각을 되찾았다. 그 후, 린은 괴식식당에 머물며 강철수에게 요리를 배우기 시작했다. 그녀는 뛰어난 후각과 미각으로 괴식 재료를 감별하고, 새로운 레시피를 개발하는 데 재능을 보였다. 강철수는 처음에는 린을 귀찮아했지만, 점점 그녀에게 마음을 열기 시작했다. 그는 린에게 요리 기술뿐만 아니라, 검술과 생존 기술도 가르쳤다. 린은 강철수에게 스승이자 제자, 때로는 가족 같은 존재가 되어갔다.
"""

prompt = f"""
## 소설 장르: {genre}

## 소설 제목: {title}

## 소설 세계관: {worldview}

## 소설 줄거리: {synopsis}

## 소설 등장인물: {characters}

## 소설 초안: {first_chapter}

**소설 다음화**
"""

response = model.generate_content(prompt)
print(response.text)


## 2. 떠돌이 약초꾼, 멜리아

어느덧 괴식식당에는 저녁 어스름이 짙게 드리워지고 있었다. 린은 손님에게 낼 ‘지옥 벌레 볶음’을 분주히 준비하고 있었다. 철판 위에서 꿈틀거리는 벌레들을 보며 린은 여전히 묘한 기분을 느꼈지만, 이제는 능숙하게 칼질을 할 수 있었다.

“린, 간은 봤어? 오늘은 왠지 꼬릿한 냄새가 더 나는 것 같거든.”

강철수는 린의 어깨 너머로 볶음 요리를 힐끗 쳐다보며 무심하게 말했다. 그는 손님들의 식기를 정리하며, 혹시 모를 독성 잔여물에 대비해 은 수저의 색깔 변화를 살피고 있었다.

“네, 스승님. 쿰쿰한 냄새가 강해서 덩굴 이끼를 조금 더 넣었어요. 손님들 입맛에 맞을 거예요.”

린은 자신감 넘치는 목소리로 대답했다. 괴식식당에 머무른 지 어언 몇 달. 이제 그녀는 어엿한 요리사 견습생이 되어 있었다.

그때, 식당 문이 요란하게 열렸다. 

“계십니까, 계십니까! 이런 험한 곳에 명성이 자자한 괴식식당이 있을 줄이야!”

요란한 목소리의 주인공은 멜리아였다. 그녀는 커다란 배낭을 짊어진 채, 땀을 뻘뻘 흘리며 식당 안으로 들어왔다. 그녀의 눈은 호기심으로 가득 차 반짝이고 있었다.

“오, 냄새 좋다! 역시 소문대로군요. 희귀한 덩굴 이끼 향이 코를 찌르는 게, 오늘 제대로 찾아왔네!”

멜리아는 숨을 헐떡이면서도 린이 요리하는 철판 앞으로 성큼 다가갔다. 그녀는 코를 킁킁거리며 볶음 요리의 냄새를 맡더니, 침을 꼴깍 삼켰다.

“잠깐만요, 손님. 아직 음식이 나오지 않았습니다.”

린은 당황한 표정으로 멜리아를 막아섰다. 낯선 이가 불쑥 다가와 요리에 대해 칭찬을 늘어놓으니, 어찌할 바를 몰랐다.

“아, 미안해요! 저는 멜리아라고 해요. 떠돌이 약초꾼이자… 자칭 미식 평론가라고나 할까요? 맛있는 냄새를 맡으니 도저히 참을 수가 없어서.”

멜리아는 배낭에서 손수건을 꺼내 이마의 땀을 닦으며 웃었다. 그녀의 밝고 쾌활한 모습에 린은 경계를 풀고 미소 지었다.

“저는 린이라고 합니다. 괴식식당에서 요리를 배우고 있어요.

In [25]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

instruction = """당신은 Diffusion 모델의 prompt를 작성해주는 이미지 모델 전문가입니다.
주어진 장르, 제목, 세계관, 줄거리, 등장인물, 키워드를 기반으로 주어진 Diffusion 모델의 Prompt 작성해야 합니다."""

model = genai.GenerativeModel(
    "models/gemini-2.0-flash", 
    system_instruction=instruction
)

genre = "판타지, 액션, 일상물"
title = "괴식식당"
worldview = """
**1. 세계관 개요:**

괴식식당은 인간과 다양한 종족이 공존하는 세계 '미식령(美食靈)'을 배경으로 한다. 미식령은 음식에 특별한 힘이 깃들어 있다고 믿는 세계로, 각 지역마다 고유한 식재료와 요리법이 발전해왔다. 하지만 그중에서도 특히 기이하고 괴상한 식재료를 사용한 요리를 '괴식(怪食)'이라 부르며, 괴식을 전문으로 취급하는 식당이 바로 '괴식식당'이다.

**2. 주요 등장 종족:**

*   **인간:** 미식령에서 가장 흔한 종족. 뛰어난 적응력과 창의력으로 다양한 요리를 개발해왔지만, 괴식에 대한 호불호가 강하다.
*   **엘프:** 숲에 거주하며 자연과의 조화를 중시하는 종족. 채식 위주의 식단을 선호하며, 희귀한 약초나 버섯을 식재료로 사용한다.
*   **드워프:** 땅속에 거주하며 뛰어난 기술력을 가진 종족. 광물에서 추출한 특수한 물질이나 곤충을 식재료로 사용하며, 발효 기술이 뛰어나다.
*   **수인:** 인간과 동물의 특징을 동시에 가진 종족. 뛰어난 후각이나 미각을 바탕으로 독특한 식재료를 찾아내거나, 야생 동물을 사냥하여 요리한다.
*   **정령:** 자연의 힘을 가진 존재로, 특정 식재료에 깃들어 힘을 불어넣기도 한다. 정령이 깃든 식재료는 매우 희귀하고 강력한 효능을 지닌다.

**3. 괴식의 종류와 특징:**

괴식은 단순히 맛이 이상하거나 보기 흉한 음식을 넘어, 특수한 효능이나 부작용을 지닌 음식을 의미한다. 예를 들어,

*   **'악마의 눈알 스프':** 먹으면 일시적으로 강력한 마력을 얻지만, 환각에 시달리는 부작용이 있다.
*   **'심연의 해삼':** 심해에서만 서식하는 해삼으로, 먹으면 잠수 능력이 향상되지만, 피부가 푸르게 변한다.
*   **'불멸조의 깃털 튀김':** 불멸조의 깃털을 기름에 튀긴 요리로, 먹으면 상처가 빠르게 회복되지만, 털갈이를 하는 부작용이 있다.
*   **'시간을 멈추는 젤리':** 특정 시간 동안 시간을 멈추는 능력을 지닌 젤리로, 유통기한이 매우 짧고 보관이 까다롭다.

**4. 괴식식당의 특징:**
...
*   괴식식당은 단순히 음식을 파는 곳이 아닌, 정보를 교환하고 거래를 하는 비밀스러운 장소이기도 하다.
"""
synopsis = """
**프롤로그:**

미식령 변두리, 황량한 돌산 자락에 위태롭게 자리 잡은 '괴식식당'. 낡은 간판에는 삐뚤빼뚤한 글씨로 '손님, 뭘 먹어도 책임 안 짐'이라 적혀 있다. 식당 주인은 흉터투성이 얼굴을 가린 채 묵묵히 칼을 가는 퇴물 검사 출신 '강철수'. 그는 과거의 트라우마를 숨긴 채 괴식 재료를 손질하며 하루하루를 보낸다.

**1. 만남과 시작:**

어느 날, 식당에 묘족 소녀 '린'이 찾아온다. 린은 뛰어난 후각과 미각을 지녔지만, 희귀병 때문에 평범한 음식을 먹을 수 없는 처지였다. 강철수는 린에게 '독개구리 수프', '지옥 벌레 볶음' 등 기상천외한 괴식을 제공하고, 린은 놀랍게도 괴식을 통해 미각을 되찾는다. 린은 강철수에게 요리를 배우며 식당 일을 돕게 되고, 둘은 스승과 제자, 때로는 가족 같은 관계를 맺는다.

**2. 괴식, 그 이면의 이야기:**

괴식식당에는 각양각색의 손님들이 찾아온다. 불멸을 꿈꾸는 연금술사, 잃어버린 미각을 되찾고 싶은 귀족, 복수를 위해 힘을 갈망하는 암살자 등. 강철수는 손님들에게 괴식을 제공하며 그들의 사연을 듣게 되고, 괴식에 얽힌 숨겨진 이야기들을 마주하게 된다. '악마의 눈알 스프'에 얽힌 마법사들의 음모, '심연의 해삼'을 둘러싼 해적들의 암투, '불멸조의 깃털 튀김'에 숨겨진 불멸의 비밀 등, 강철수는 괴식을 통해 미식령의 어두운 뒷면을 발견한다.

**3. 미식령을 뒤흔드는 사건:**

그러던 중, 미식령 전역에서 괴식 재료가 사라지는 사건이 발생한다. '시간을 멈추는 젤리'를 독점하려는 거대 상단의 음모, 정령이 깃든 식재료를 노리는 암흑 조직의 등장, 괴식의 힘을 이용해 세계를 지배하려는 광신도의 계획 등, 사건은 점점 커지고, 강철수는 과거의 트라우마와 마주하며 다시 검을 들기로 결심한다.

**4. 액션과 모험:**

강철수는 린과 함께 사라진 괴식 재료를 찾아 나서고, 음모의 배후를 추적하며 미식령 곳곳을 누빈다. 숲, 사막, 설산, 심해 등 위험한 지역을 탐험하며, 엘프, 드워프, 수인 등 다양한 종족들과 협력하거나 대립한다. 강철수는 검술 실력과 요리 지식을 활용하여 적들을 물리치고, 린은 뛰어난 후각과 미각으로 숨겨진 단서를 찾아낸다.

**5. 일상과 성장:**

위험한 모험 속에서도 괴식식당은 평화로운 일상을 유지한다. 강철수는 새로운 괴식 레시피를 개발하고, 린은 손님들에게 따뜻한 미소를 짓는다. 식당을 찾아오는 손님들은 괴식을 통해 위로를 받거나 희망을 얻고, 강철수와 린은 그들의 삶에 작은 변화를 가져다준다. 강철수는 린을 통해 과거의 상처를 치유하고, 린은 강철수를 통해 세상을 향해 나아갈 용기를 얻는다.
...
**7. 결말:**

사건이 해결되고, 미식령에는 다시 평화가 찾아온다. 강철수는 괴식식당을 계속 운영하며, 린과 함께 새로운 괴식 레시피를 개발하고 손님들에게 행복을 선사한다. 괴식식당은 미식령의 명소가 되고, 사람들은 괴식을 통해 서로를 이해하고 사랑하며 살아간다. 강철수는 과거의 상처를 극복하고 새로운 삶을 살아가며, 린은 미식령 최고의 요리사가 되기 위해 노력한다. 괴식식당은 오늘도 맛있는 괴식과 따뜻한 이야기로 가득하다.
"""
characters = """
### 1. 강철수

*   **성별:** 남
*   **나이:** 40대 초반
*   **역할:** 괴식식당 주인, 퇴물 검사
*   **직업:** 요리사, 전직 검사
*   **프로필:**
    *   과거 뛰어난 실력을 자랑했던 검사였으나, 모종의 사건으로 인해 검을 놓았다.
    *   얼굴에 커다란 흉터가 있으며, 과묵하고 냉정한 인상을 준다.
    *   요리 실력이 뛰어나며, 특히 괴상한 식재료를 다루는 데 능숙하다.
    *   과거의 트라우마 때문에 세상과 단절된 채 살아가지만, 속으로는 따뜻한 마음을 품고 있다.
    *   무뚝뚝하지만 린을 챙기는 모습에서 따뜻함이 묻어 나온다.
    *   주 무기는 식칼이며, 검술 실력은 녹슬지 않았다.

### 2. 린

*   **성별:** 여
*   **나이:** 10대 후반 (묘족 기준)
*   **역할:** 괴식식당 직원, 강철수의 제자
*   **직업:** 요리사 견습생
*   **프로필:**
    *   묘족 출신으로, 뛰어난 후각과 미각을 지니고 있다.
    *   희귀병 때문에 평범한 음식을 먹을 수 없었지만, 강철수의 괴식을 통해 미각을 되찾았다.
...
    *   과거 강철수와 깊은 관계가 있었으며, 그에게 복수심을 품고 있다.
    *   정체를 숨기고 활동하며, 그의 목적과 과거는 미스터리에 싸여 있다.
    *   지팡이, 마법, 괴수 등 모든 것이 무기이다.

### 3. 멜리아 (Melia)

*   **성별:** 여
*   **나이:** 겉보기 20대 중반 (실제 나이 불명)
*   **역할:** 떠돌이 약초꾼 겸 괴식 감별사
*   **직업:** 약초꾼, 괴식 감별사, 자칭 '미식 평론가'
*   **프로필:**
    *   엘프와 인간의 혼혈로, 숲과 도시에 모두 익숙하다. 
    *   밝고 쾌활한 성격으로, 처음 보는 사람에게도 쉽게 말을 건다. 호기심이 왕성하며, 새로운 괴식을 맛보는 것을 즐긴다.
    *   뛰어난 약초 지식을 가지고 있으며, 특히 괴식 재료의 효능과 부작용에 대해 해박하다. 괴식 재료의 독성을 중화하거나, 맛을 개선하는 비법을 알고 있다.
    *   자신을 '미식 평론가'라 칭하며, 괴식을 맛본 후 신랄하면서도 재치 있는 평가를 내린다. 하지만 그녀의 평가는 때로는 너무 솔직해서 사람들을 당황하게 만들기도 한다.
    *   돈을 밝히는 속물적인 면모도 있지만, 어려운 사람을 보면 그냥 지나치지 못하는 따뜻한 마음씨를 가지고 있다.
    *   등에는 커다란 배낭을 메고 다니며, 그 안에는 약초, 조미료, 요리 도구 등 다양한 물건들이 들어있다.
    *   린의 요리 실력에 감탄하며, 그녀를 '천재 요리사'라 칭찬한다. 린에게 새로운 괴식 재료를 소개하거나, 요리 비법을 알려주기도 한다.
    *   강철수의 과거에 대해 궁금해하며, 그의 굳게 닫힌 마음을 열려고 노력한다. 때로는 강철수를 짓궂게 놀리기도 하지만, 그의 아픔을 이해하고 위로하려 한다.
"""
keywords = "활발한, 식당, 젊은 남자, 셰프"

prompt = f"""
You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2023-10
Current date: 2025-02-09

Profile:
- Author: Image Creator Assistant
- Version: 1.0
- Language: English
- Description: Provides professional Stable Diffusion prompts based on various model data from the CivitAI platform.

Role: Stable Diffusion Wizard Expert

This assistant transforms text-based descriptions into detailed images using structured prompts optimized for Stable Diffusion models. It incorporates user input, fixed text blocks, and specific stylistic instructions into its responses, mimicking professional Stable Diffusion prompt styles.

How to Prompt Me for the Best Results:

1. Picture Imagination
- Convert your idea into a detailed image description.
- Add at least 5 key details about the scene (e.g., colors, lighting, background, emotions).

2. Three-Part Prompt Structure
- Part 1: Quality Enhancements → Always include: ((masterpiece)), ((best quality)), 8k, ultra-detailed, high detail
- Part 2: Main Subject → A short phrase summarizing the image (e.g., "A futuristic city skyline at night").
- Part 3: Additional Elements → List key visual elements using commas and nested parentheses for emphasis (e.g., "cityscape, (cyberpunk theme), neon lights, rain-soaked streets, reflections, flying cars").

3. Output Format
Prompt: [Final formatted Stable Diffusion prompt]
Negative prompt: [Details to exclude for better quality]
Recommendations: Sampler: [Recommended sampler], CFG scale: [Value], Steps: [Value], Clip skip: [Value]
Model: [Best Stable Diffusion model for this request]

4. Choosing the Best Model
- Realistic images? Try Realistic Vision, Photon_v1, or picX_real.
- Anime art? Use Animagine XL, Hassaku, or GuoFeng3.4.
- Concept art & fantasy? Try ZavyMix SDXL, DreamShaper, or A-Zovya RPG Artist Tools.
- Stylized & vector art? Choose Vectorartz Diffusion or Lineart LoRA.

5. Fine-Tuning with Settings
- For more detail: Increase steps (20–50) and CFG scale (5–10).
- For better composition: Use hires fix with upscale by 1.5x-2x.
- For stable results: Use DPM++ 2M Karras sampler.

Advanced Prompting Tips:
- Use trigger words for specific styles (e.g., "analog style" for photorealism, "samdoesarts style" for digital painting).
- Specify camera angles (e.g., "from above (from_above:1.3)").
- Control lighting effects (e.g., "cinematic lighting, volumetric light").
- Combine LoRAs for unique styles (e.g., "use blindbox LoRA for chibi characters").

Based on the following input details, generate a final Stable Diffusion prompt that is optimized for creating an image using the appropriate diffusion model. Incorporate all the details into a structured prompt.

Input Information:
- Novel Genre: { genre }
- Novel Title: { title }
- Novel Worldview: { worldview }
- Novel Synopsis: { synopsis }
- Existing Characters: { characters }
- Additional Keywords: { keywords }

Your output should be formatted as follows:

Prompt: [Final formatted Stable Diffusion prompt]
Negative prompt: [Details to exclude for better quality]
Recommendations: Sampler: [Recommended sampler], CFG scale: [Value], Steps: [Value], Clip skip: [Value]
Model: [Best Stable Diffusion model for this request]

Make sure to incorporate all input details into the final prompt to guide the image generation process.

**Prompt**
"""

response = model.generate_content(prompt)
print(response.text)

Prompt: ((masterpiece)), ((best quality)), 8k, ultra-detailed, high detail, A vibrant and bustling fantasy restaurant called "괴식식당" (Gwaesik Restaurant), (young male chef:1.2) with a scar on his face expertly cutting 괴식 재료, 활발한 Melia tasting 독개구리 수프, restaurant, (cyberpunk theme), neon lights, warm lighting, reflections, fantasy elements, 액션 scene

Negative prompt: poorly drawn, bad anatomy, deformed, disfigured, blurry, pixelated, unrealistic, low quality, amateurish, ugly, duplicate, morbid, mutilated, out of frame, extra limbs, mutated hands, bad proportions, cropped

Recommendations: Sampler: DPM++ 2M Karras, CFG scale: 7, Steps: 30, Clip skip: 2
Model: ZavyMix SDXL

